In [459]:
import torch.nn as nn
import os
import pandas as pd
import numpy as np
import pickle

In [460]:
!ls data/ml-1m

movies.dat  ratings.dat  README  users.dat


In [461]:
# data_dir = os.path.join(os.getcwd(),"data","ml-100k")
data_dir = os.path.join(os.getcwd(),"data","ml-1m")

In [462]:
data_dir

'/arnavvolume/RecNet/data/ml-1m'

In [463]:
data_arr = []

In [464]:
with open(os.path.join(data_dir,"ratings.dat")) as f:
    data = f.readlines()

In [465]:
data[0]

'1::1193::5::978300760\n'

In [466]:
items_arr = []

In [467]:
with open(os.path.join(data_dir,"movies.dat"),encoding="ISO-8859-1") as f:
    items = f.readlines()

In [468]:
len(items)

3883

In [469]:
for it in items:
    split = str(it).split("::")[0:2]
#     print(split)
    temp = [int(split[0]),split[1]]
    items_arr.append(temp)

In [470]:
item_df = pd.DataFrame(data=items_arr,columns=["id","item"])
# item_df = pd.read_csv(os.path.join(data_dir,"movies.csv"))

In [471]:
new_item_df[item_df["id"]==100]

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,id,item
98,99,City Hall (1996)


In [472]:
for row in data:
    data_arr.append(row.strip().split("::"))

In [473]:
data_arr = np.array(data_arr)

In [474]:
data_df = pd.DataFrame(data=data_arr,columns=["user_id","item_id","rating","timestamp"])

## Find all users and items having count and interactions lesser than 5

In [475]:
rem_items = data_df['item_id'].value_counts()[data_df['item_id'].value_counts()<=5].index

In [476]:
len(data_df)

1000209

In [477]:
new_data_df = data_df[~data_df['item_id'].isin(rem_items)]

In [478]:
len(item_df)

3883

In [479]:
new_item_df=item_df[~item_df['id'].isin(list(map(lambda x:int(x),rem_items)))]

In [480]:
len(new_item_df)

3554

In [481]:
new_data_df = new_data_df.sort_values(['user_id', 'timestamp'], ascending=[1, 1])

In [482]:
new_old = {}
old_new = {}
new_old_str = {}
old_new_str = {}

In [483]:
for i,row in new_item_df.iterrows():
    new_old[i+1] = row[0]
    old_new[row[0]] = i+1
    new_old_str[str(i+1)] = str(row[0])
    old_new_str[str(row[0])] = str(i+1)

In [484]:
new_item_df.id = new_item_df.id.map(old_new)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [485]:
new_data_df.item_id = new_data_df.item_id.map(old_new_str)

In [486]:
# data_df = pd.read_csv(os.path.join(data_dir,"ratings.csv"))
# data_df = data_df.sort_values(['userId', 'timestamp'], ascending=[1, 1])

In [487]:
step_size= 200
max_len = 200

In [488]:
len(new_data_df)

999416

In [489]:
new_data_df.head()

,user_id,item_id,rating,timestamp
31,1,3118,4,978300019
22,1,1251,5,978300055
27,1,1673,4,978300055
37,1,1010,5,978300055
24,1,2272,3,978300103


In [490]:
new_item_df.head()

,id,item
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [491]:
unique_user_id,indices,counts = np.unique(new_data_df['user_id'].values,
                                          return_index=True,
                                          return_counts=True)

In [492]:
count=0

In [493]:
def sliding_window(item_ids,step_size,max_len):
    global count
#     print(item_ids)
    
    for i in range(len(item_ids),0,-step_size):
#         if len(item_ids[max(i-max_len,0):i])>=max_len:
        if len(item_ids[max(i-max_len,0):i])>10:
#             count+=1
            yield item_ids[max(i-max_len,0):i]

In [494]:

def return_sequences(df,indices,unique_user_id,max_len,step_size):
    
    for idx,start_index in enumerate(indices):
        if idx==len(indices)-1:
            break
        user_id=unique_user_id[idx]

        last_index = indices[idx+1]

        
        for seq in sliding_window(df['item_id'].values[start_index:last_index],step_size,max_len) :
            
            yield (user_id,seq)
            
       

In [495]:
seq_arr = []
for seq in return_sequences(new_data_df,indices,unique_user_id,max_len,step_size):
    seq_arr.append(seq)

In [458]:
count

0

In [496]:
# !rm sequences.pkl
!touch sequences_1m_200.pkl

In [497]:
with open("sequences_1m_200.pkl","wb") as f:
    pickle.dump(seq_arr,f)

In [498]:
arr = pickle.load(open("sequences_1m_200.pkl","rb"))

In [499]:
arr[:400]

[('1', array(['3118', '1251', '1673', '1010', '2272', '1769', '3340', '2736',
         '1190', '1177', '712', '258', '908', '605', '2624', '1893', '1960',
         '3037', '927', '1023', '1894', '1950', '149', '1016', '1082',
         '903', '1268', '2729', '2694', '1227', '656', '2850', '528',
         '3046', '2723', '2253', '1017', '1180', '591', '2330', '1507',
         '524', '592', '2619', '736', '585', '1', '2287', '2226', '774',
         '1527', '1839', '48'], dtype=object)),
 ('10',
  array(['315', '771', '3191', '1704', '1008', '3184', '583', '2014', '290',
         '2385', '2043', '2022', '2070', '540', '251', '3433', '330',
         '3004', '2355', '1978', '1975', '2041', '1631', '1109', '2026',
         '326', '2363', '2065', '1004', '2056', '2368', '1591', '927',
         '1325', '1360', '341', '367', '1534', '1986', '1120', '1937',
         '1979', '432', '1351', '348', '1110', '1934', '1387', '2025',
         '1981', '2248', '2356', '820', '280', '1076', '1816', '3085',

In [500]:
item_to_ix = {}
ix_to_item = {}

In [501]:
for i,row in new_item_df.iterrows():
    item_to_ix[row[1]] = row[0]
    ix_to_item[row[0]] = row[1]

In [502]:
!touch ix_to_item.pkl
!touch item_to_ix.pkl

In [503]:
with open("ix_to_item.pkl","wb") as f:
    pickle.dump(ix_to_item,f)

In [504]:
with open("item_to_ix.pkl","wb") as f:
    pickle.dump(item_to_ix,f)

In [505]:
test = pickle.load(open("item_to_ix.pkl","rb"))